
Orchestrating molecules and crystals using ASE
==========

- **Complexity level**: intermediate
- **Requirements**: basic knowledge of crystallography and Python



In [ ]:
!pip install ase
import ase
ase.__version__


Which ASE version was installed? Note, you can also install specific version of a package with: `!pip install some_nice_package==0.42.0`

Let's continue.


In [ ]:
from io import StringIO
from ase import Atom, Atoms
from ase.spacegroup import crystal
from ase.calculators.emt import EMT
from ase.optimize import BFGS
from ase.io.vasp import read_vasp
from ase.io import write
from ase.build import surface
from ase.geometry import cell_to_cellpar
import spglib  # Gotcha here :-) a reader is expected to fix an error


These are all the imports we need today.

Let's build a non-periodic molecule and then create a periodic structure from it.


In [ ]:
molecule = Atoms('OCCHHHHHH', positions=[
    [-1.1712,  0.2997,  0     ],         
    [-0.0463, -0.5665,  0     ],         
    [ 1.2175,  0.2668,  0     ],         
    [-0.0958, -1.212,   0.8819],         
    [-0.0952, -1.1938, -0.8946],         
    [ 2.105,  -0.372,  -0.0177],         
    [ 1.2426,  0.9307, -0.8704],         
    [ 1.2616,  0.9052,  0.8886],         
    [-1.1291,  0.8364,  0.8099],         
])                                       
periodic_molecule = Atoms([atom for atom in molecule], cell=(12, 12, 12), pbc=True)


We have now `molecule` and `periodic_molecule`. What is the difference?

Let's visualize what we have built.


In [ ]:
write('-', molecule, format='vasp')  # Gotcha here :-) a reader is expected to fix an error


Here are some more functions we might use.

This is `refine`:


In [ ]:
def refine(ase_obj, accuracy=1E-03, conventional_cell=False):                    
    try:                                                                         
        symmetry = spglib.get_spacegroup(ase_obj, symprec=accuracy)              
        lattice, positions, numbers = spglib.standardize_cell(ase_obj,           
            symprec=accuracy, to_primitive=not conventional_cell)                
    except:                                                                      
        return None, 'Error while structure refinement'                          
    try:                                                                         
        spacegroup = int( symmetry.split()[1].replace('(', '').replace(')', '') )
    except (ValueError, IndexError, AttributeError):                             
        return None, 'Symmetry error (coinciding atoms?) in structure'           
    try:                                                                         
        return crystal(                                                          
            Atoms(                                                               
                numbers=numbers,                                                 
                cell=lattice,                                                    
                scaled_positions=positions,                                      
                pbc=True                                                         
            ),                                                                   
            spacegroup=spacegroup,                                               
            primitive_cell=not conventional_cell,                                
            onduplicates='replace'                                               
        ), None                                                                  
    except:                                                                      
        return None, 'Unrecognized sites or invalid site symmetry in structure'    


This is `poscar_to_ase`:


In [ ]:
def poscar_to_ase(poscar_string):                                 
    ase_obj, error = None, None                                   
    buff = StringIO(poscar_string)                                
    try:                                                          
        ase_obj = read_vasp(buff)                                 
    except AttributeError:                                        
        error = 'Types of atoms can be neither found nor inferred'
    except Exception:                                             
        error = 'Cannot process POSCAR: invalid or missing data'  
    buff.close()                                                  
    return ase_obj, error                                           


Let's have a look what we can do more:


In [ ]:
crystal_obj.get_chemical_formula()
crystal_obj *= (2, 2, 2)
obj2d = surface(crystal_obj, (0, 0, 1), 5, periodic=True)


Try to write a function calculating a volume of the primitive cell for any ASE crystalline structure. It is given by the determinant of the cell parameters 3x3 matrix. In ASE it's a `.cell` attribute of your crystal structure object.

Hint: you'll need a `det` function from the `numpy.linalg` Python package.


In [ ]:
# use this cell for your custom Python code
# copy and paste your code here from the slides and the other cells